<a href="https://colab.research.google.com/github/SaraGhR/hip-fracture-care-episodes/blob/main/hip_fracture_analyzing_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>